In [127]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import numpy as np
import pickle

In [128]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [129]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [130]:
data['Gender']

0       Female
1       Female
2       Female
3       Female
4       Female
         ...  
9995      Male
9996      Male
9997    Female
9998      Male
9999    Female
Name: Gender, Length: 10000, dtype: object

In [131]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [132]:
data['Gender']

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64

In [133]:
#one hot encoding for 'Geography' column
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
geography_encoded = onehot_encoder.fit_transform(data[['Geography']]).toarray()
geography_df = pd.DataFrame(geography_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))
onehot_encoder.get_feature_names_out(['Geography'])


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [134]:
geography_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [135]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [136]:
data = pd.concat([data.drop('Geography', axis=1), geography_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [137]:
# Split the data into features and target variable
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [138]:
#Save the encoders and scaler for future use
with open('label_encoder_gender_r.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)    
with open('onehot_encoder_geography_r.pkl', 'wb') as f:
    pickle.dump(onehot_encoder, f)


In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   

X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,0,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,0,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,1,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,1,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,1,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,0,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,0,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,1,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,1,1.0,0.0,0.0


In [140]:
## Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [141]:
## ANN Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [142]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), #HL1 connect with input layer
    Dense(32, activation='relu'), #HL2
    Dense(1), #Output Layer
])
## Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [143]:


## compile the model
model.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae'])

In [144]:
## setup TensorBoard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
## Setup EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [145]:
### Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping, tensorboard_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 13380704256.0000 - mae: 100363.5781 - val_loss: 12998080512.0000 - val_mae: 98472.3828
Epoch 2/100
250/250 [==============================] - 0s 972us/step - loss: 13208279040.0000 - mae: 99505.7188 - val_loss: 12660768768.0000 - val_mae: 96765.5781
Epoch 3/100
250/250 [==============================] - 0s 919us/step - loss: 12614501376.0000 - mae: 96548.6875 - val_loss: 11811025920.0000 - val_mae: 92462.3203
Epoch 4/100
250/250 [==============================] - 0s 849us/step - loss: 11452709888.0000 - mae: 90793.7344 - val_loss: 10400316416.0000 - val_mae: 85330.6484
Epoch 5/100
250/250 [==============================] - 0s 834us/step - loss: 9781879808.0000 - mae: 82469.0078 - val_loss: 8597936128.0000 - val_mae: 76285.9844
Epoch 6/100
250/250 [==============================] - 0s 839us/step - loss: 7870337536.0000 - mae: 72888.4766 - val_loss: 6747623936.0000 - val_mae: 67036.8594
Epoch 7/100
250/250 [======

In [146]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [147]:
#tensorboard --logdir logs/fit

In [148]:
#evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae}")

63/63 [==============================] - 0s 553us/step - loss: 3342698496.0000 - mae: 49995.3398
Test MAE: 49995.33984375


In [149]:
model.save('salary_regression_model.h5')

/Users/majidkhan/Projects/Gen_AI/ANN_Classification/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
